In [1]:
import numpy as np
import os
import cv2
import mediapipe as mp
import subprocess
mp_selfie_segmentation = mp.solutions.selfie_segmentation
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
print(cv2.getBuildInformation())


General configuration for OpenCV 4.8.0 =====================================
  Version control:               4.8.0

  Platform:
    Timestamp:                   2023-06-30T11:22:01Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (7 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (35 files):           + SSSE3 SSE4_1 

In [3]:
video_directory = "E:\\course_work\\Dissertation\\back_001"

all_files_in_directory = os.listdir(video_directory)

video_files = [file for file in all_files_in_directory if file.endswith(('.mkv'))]

videos = []

for video_file in video_files:
    video_file_path = os.path.join(video_directory, video_file)
    output_file_path = os.path.splitext(video_file_path)[0] + '.mp4'
    subprocess.run(['ffmpeg', '-i', video_file_path, output_file_path])
    videos.append(output_file_path)


In [4]:
  
output_folder = "E:\\course_work\\Dissertation\\pose_estim" 
output_extension = ".mp4"

# Set the video writer parameters
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Specify the video codec (e.g., "mp4v" for MP4)

frame_size = (1280, 720)


<img src="https://www.mdpi.com/applsci/applsci-13-02700/article_deploy/html/images/applsci-13-02700-g001.png" style="height=200px" >

In [5]:
import time
with mp_pose.Pose(min_detection_confidence=0.7 , min_tracking_confidence=0.7) as pose,mp_selfie_segmentation.SelfieSegmentation(model_selection=0) as selfie_segmentation:
    for vid in videos:
        video = cv2.VideoCapture(vid)
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

        filename = os.path.splitext(os.path.basename(vid))[0]
        output_path = os.path.join(output_folder, filename + output_extension)
        original_fps = video.get(cv2.CAP_PROP_FPS)
        original_frame_size = (
            int(video.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        )
        video_writer = cv2.VideoWriter(output_path, fourcc, original_fps, frame_size)
        while True:
            ret,frame = video.read()
            if not ret:
                break

            resized_frame = cv2.resize(frame, frame_size)
            image = cv2.cvtColor(resized_frame,cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            segmentation_results = selfie_segmentation.process(image)
            image.flags.writeable = True
            
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            condition = np.stack((segmentation_results.segmentation_mask,) * 3, axis=-1) > 0.1
            # this two will blurred background 
            # blurred_image = cv2.GaussianBlur(image, (99, 99), 30)
            # output_image = np.where(condition, image, blurred_image)

            # this below lines change background to green color
            background_color = np.zeros_like(image)
            background_color[:] = [0, 255, 0]  # Green in BGR
            output_image = np.where(condition, image, background_color)
            

            video_writer.write(output_image)
        
        video.release()
        video_writer.release()

